# 固有値解析
### Agenda

- PageRankアルゴリズムで用いられた固有値、固有ベクトルをレビューする

### Clean slate

In [1]:
## create the clean environment
import gc
import matplotlib.pyplot as plt

def clear_all():
    # Clears all the variables from the workspace
    gl = globals().copy()
    for var in gl:
        if var in clean_env_var: continue
        del globals()[var]
    # Garbage collection:
    gc.collect()

def close_plots():
  my_plots = plt.get_fignums()
  for j in my_plots:
    plt.close(plt.figure(j))

clean_env_var = dir()
clean_env_var.append('clean_env_var')

In [2]:
clear_all()

### Hardware

In [3]:
%%bash
system_profiler SPHardwareDataType | grep -E \
"Model Identifier"\|"Processor Name"\|"Processor Speed"\
\|"Number of Processors"\|"Memory:"

      Model Identifier: MacBookPro13,1
      Processor Name: Dual-Core Intel Core i5
      Processor Speed: 2 GHz
      Number of Processors: 1
      Memory: 16 GB


### Python

In [4]:
!python -V

Python 3.7.4


### Import

In [5]:
import numpy as np

## 1. 行列の固有値

$$
\mathbf A\mathbf x = \lambda \mathbf x
$$

- $\mathbf A$: matrix
- $\lambda$: eigenvalue
- $\mathbf x$: eigenvector

<img src = "./fig/eigenvector_1.jpg">

### REMARKS

- 実務上重要なのは最大・最小固有値（see [PageRank.ipynb](./PageRank.ipynb)）



In [6]:
A = np.array([[1, -1],
              [-1, 2]]
             , dtype = np.float)
A

array([[ 1., -1.],
       [-1.,  2.]])

In [7]:
v, w = np.linalg.eig(A)
v

array([0.38196601, 2.61803399])

## 2. べき乗法(Power Method)

<img src = "./fig/eigenvector_2.jpg">
<img src = "./fig/eigenvector_3.jpg">
<img src = "./fig/eigenvector_4.jpg">
<img src = "./fig/eigenvector_5.jpg">
<img src = "./fig/eigenvector_6.jpg">

### REMARKS

- 重根が存在する場合、収束しないことがわかる
- 現在認められている最適の方策は，ハウスホルダー(Householder)変換で行列を単純な三重対角化行列に変形してから，反復法で解を追い込んでいくやり方

<img src = "./fig/eigenvector_7.jpg">

### 3. Power MethodとNumpyの解の近似性を確かめる

In [8]:
np.random.seed(42)
N = 10
A = np.random.randint(1, 10, (N, N))
v, w = np.linalg.eig(A)
max_index = np.argmax(abs(v))
min_index = np.argmin(abs(v))

In [9]:
w.T[max_index]

array([0.34378677+0.j, 0.32196125+0.j, 0.27634261+0.j, 0.33618891+0.j,
       0.24971248+0.j, 0.24791024+0.j, 0.38099626+0.j, 0.36501253+0.j,
       0.31742816+0.j, 0.29290723+0.j])

In [10]:
w.T[min_index]

array([-0.08645534+0.j, -0.47794872+0.j,  0.01792275+0.j,  0.35808689+0.j,
       -0.61514336+0.j, -0.11707273+0.j,  0.18258893+0.j,  0.44342927+0.j,
       -0.04146081+0.j,  0.10839023+0.j])

In [11]:
def power_method(A, init, iteration = 2000, eps = 1e-5):
    def update_rule(x):
        y = A @ x
        x = y/np.linalg.norm(y, ord=2)
        return x
    
    i = 0
    x_0 = init
    x_1 = update_rule(x = init)
    while np.allclose(x_1, x_0, atol = eps, rtol = 0) and i < iteration:
        i += 1
        x_0, x_1 = x_1, update_rule(x_1)
    
    return x_1
        
        

In [12]:
power_method(A = A, init = np.ones(N))

array([0.34219629, 0.31269661, 0.28319693, 0.33629635, 0.25959719,
       0.25959719, 0.37169597, 0.36579603, 0.31859655, 0.28909687])

計算精度は低いので素直に`numpy.linalg`を用いた方が良い